# Franka Robot Grasping Tutorial: With and Without Spinning Pads

This tutorial demonstrates a comparison between two Franka Emika Panda robots—one with spinning pads and one without—performing the same grasping task. The spinning pads reduce the torsional friction about the normal vector of the contact between robot's finger and the object. 
"The implementation for friction in genesis is not as advanced that in Mujoco. There is only 1 scalar parameter in Genesis, and condim cannot be changed (only condim = 3 is implemented). There is no torsional friction at the moment. The torsional friction in genesis is the result of multi-contact friction."


The standard robot uses joints 0-8, with 0-6 for arm movement and 7-8 for the gripper. The modified robot includes spinning pads at joints 9-10, which can help provide rotational slippage at the contact point

In [1]:
import genesis as gs
import numpy as np
import torch
from pytransform3d import (
    transformations as pt,
    rotations as pr,
    batch_rotations as pb,
    trajectories as ptr,
    plot_utils as ppu
)
from pandaSim.geometry.genesis_adapter import GenesisAdapter
from pandaSim.planning.screw_motion_planner import ScrewMotionPlanner


import spatialmath as sm
# auto reaload
%load_ext autoreload
%autoreload 2

[I 05/15/25 10:36:51.942 9043] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


In [2]:

gs.init(backend=gs.gpu)
gs.set_random_seed(seed=42)
adapter = GenesisAdapter()
planner = ScrewMotionPlanner()

[Genesis] [10:37:18] [INFO] ╭───────────────────────────────────────────────╮
[Genesis] [10:37:18] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│
[Genesis] [10:37:18] [INFO] ╰───────────────────────────────────────────────╯
[Genesis] [10:37:18] [INFO] Running on [NVIDIA GeForce RTX 3070 Laptop GPU] with backend gs.cuda. Device memory: 7.78 GB.
[Genesis] [10:37:18] [INFO] 🚀 Genesis initialized. 🔖 version: 0.2.1, 🌱 seed: None, 📏 precision: '32', 🐛 debug: False, 🎨 theme: 'dark'.
[Genesis] [10:37:19] [INFO] Scene <4e83542> created.


## Scene Construction

Here we create the simulation environment with:
1. A ground plane
2. Two Franka Emika Panda robots:
   - One standard robot at position (0,2,0)
   - One with spinning pads at position (0,0,0)
3. Two cubes for each robot
4. A target entity for visualization

The key difference between the robots is that the standard Franka `(franka_MJCF)` uses only the arm and finger joints (joints 0-8), while the other version has additional spinning pads (joints 9-10) that cause rotational slippage using extra hinge DoFs.


In [3]:
scene = adapter.scene
plane = scene.add_entity(
    gs.morphs.Plane(),
)

# franka_MJCF = scene.add_entity(
#     gs.morphs.MJCF(file="xml/franka_emika_panda/panda.xml",
#                    pos = (0.0, 2.0, 0.0)),
    
# )
franka = scene.add_entity(
    gs.morphs.MJCF(file="../assets/xml/franka_emika_panda/panda_vir.xml")
                   
)
# franka_virtual = scene.add_entity(
#     gs.morphs.MJCF(file="../assets/xml/franka_emika_panda/panda.xml",
#                    pos = (0, -2, 0))
# )
cube_size = (0.1, 0.07, 0.25)
cube_pos = [(0.7, 0, 0.05), (0.7, 2.0, 0.05)]

cube, cube2 = [scene.add_entity(
    gs.morphs.Box(
        size = cube_size,
        pos  = pos,
        euler = (0, -90, 0)
    ),
    surface=gs.surfaces.Default(
        color=(0.5, 0.8, 0.94),
    ),
    material=gs.materials.Rigid(friction=5)
    
) for pos in cube_pos]

target_entity = scene.add_entity(
    gs.morphs.Mesh(
        file="meshes/axis.obj",
        scale=0.15,
        collision=False,
    ),
    surface=gs.surfaces.Default(color=(1, 0.5, 0.5, 1)),
    material=gs.materials.Rigid(gravity_compensation=1.0)
)

[Genesis] [10:37:22] [INFO] Adding <gs.RigidEntity>. idx: 0, uid: <8940102>, morph: <gs.morphs.Plane>, material: <gs.materials.Rigid>.
[Genesis] [10:37:22] [INFO] Adding <gs.RigidEntity>. idx: 1, uid: <6fec180>, morph: <gs.morphs.MJCF(file='/home/vahid/repos/pandaSim/assets/xml/franka_emika_panda/panda_vir.xml')>, material: <gs.materials.Rigid>.
[Genesis] [10:37:23] [WARNING] (MJCF) Tendon not supported
[Genesis] [10:37:23] [WARNING] (MJCF) Approximating tendon by joint actuator for `finger_joint1`
[Genesis] [10:37:23] [WARNING] (MJCF) Actuator control gain and bias parameters cannot be reduced to a unique PD control position gain. Using max between gain and bias.
[Genesis] [10:37:23] [WARNING] (MJCF) Approximating tendon by joint actuator for `finger_joint2`
[Genesis] [10:37:23] [WARNING] (MJCF) Actuator control gain and bias parameters cannot be reduced to a unique PD control position gain. Using max between gain and bias.
[Genesis] [10:37:23] [INFO] Adding <gs.RigidEntity>. idx: 2, 

## Robot Configuration

We build the scene and define important joint groups:
- `motors_dof`: Joints 0-6 for arm movement
- `fingers_dof`: Joints 7-8 for gripper fingers
- `virtual_hand`: Joint 9 at the middle of the finger for rotational slippage (only on the `franka`)
- `spinning_pads`: Joints 9-10 for the spinning contact pads (only on the `franka`)

In [ ]:
n_envs = 0
scene.build()


In [4]:
franka

[Genesis] [10:37:27] [WARNING] This property is deprecated and will be removed in future release. Please use 'qs_idx' instead.
[Genesis] [10:37:27] [WARNING] This property is deprecated and will be removed in future release. Please use 'qs_idx' instead.
[Genesis] [10:37:27] [WARNING] This property is deprecated and will be removed in future release. Please use 'qs_idx_local' instead.
[Genesis] [10:37:27] [WARNING] This property is deprecated and will be removed in future release. Please use 'qs_idx_local' instead.
[Genesis] [10:37:27] [WARNING] This property is deprecated and will be removed in future release. Please use 'dofs_idx' instead.
[Genesis] [10:37:27] [WARNING] This property is deprecated and will be removed in future release. Please use 'dofs_idx' instead.
[Genesis] [10:37:27] [WARNING] This property is deprecated and will be removed in future release. Please use 'dof_idx_local' instead.
[Genesis] [10:37:27] [WARNING] This property is deprecated and will be removed in future

────────────────────── <gs.RigidEntity> ──────────────────────
                'n_qs': <int>: 12
              'n_dofs': <int>: 12
             'n_links': <int>: 14
             'n_geoms': <int>: 23
             'n_cells': <numpy.int64>: 1873082
             'n_verts': <int>: 1363
             'n_faces': <int>: 2634
             'n_edges': <int>: 3951
            'n_joints': <int>: 12
            'n_vgeoms': <int>: 58
            'n_vverts': <int>: 397454
            'n_vfaces': <int>: 134888
        'n_equalities': <int>: 1
               'q_end': <int>: 12
             'q_start': <int>: 0
             'is_free': <bool>: True
            'is_built': <bool>: False
             'dof_end': <int>: 12
           'dof_start': <int>: 0
                 'idx': <int>: 1
                 'sim': <gs.Simulator>
                 'uid': <gs.UID>('6fec180-cac1c48798a96a22cc262b3bd')
           'base_link': <gs.RigidLink>: <c0bf1c3>, name: 'link0', idx: 1
       'base_link_idx': <int>: 1
            

In [5]:
for joint in franka.joints:
    print(joint.name, joint.idx)

joint1 1
joint2 2
joint3 3
joint4 4
joint5 5
joint6 6
joint7 7
finger_joint1 8
finger_joint2 9
virtual_finger_joint 10
left_pad_hinge 11
right_pad_hinge 12


In [6]:
for eq in franka.equalities:
    print(eq.eq_obj1id, eq.eq_obj2id)

8 9


# DoF Index

Note: The dof index is `joint idx_local` that is different than `link idx_local`.

In [5]:
motors_dof = np.arange(7)
fingers_dof = np.arange(7, 9)
virtual_hand = 9
spinning_pads = np.arange(10, 12)

In [ ]:
for link, joint in zip(franka.links, franka.joints):
    print(link.name, link.idx_local, '\t', joint.name, joint.idx_local)

In [ ]:
right_spinning_link = franka.get_link('right_spinning_pad')
left_spinning_pad = franka.get_link('left_spinning_pad')
virtual_finger = franka.get_link('virtual_finger')

right_spinning_link.parent_idx_local, left_spinning_pad.parent_idx_local, virtual_finger.parent_idx_local

## Initial Robot Positioning

Reset the scene and set both robots to the same initial joint configuration. We'll use this "ready pose" as a starting position for our task. Notice that the standard robot has 9 DOFs (degrees of freedom), while the modified version can have 1 dof extra for virtual_finger and two extras for spinning_pads

In [ ]:
scene.reset()
qr = np.array([0.0000, -0.3002, 0.0000, -2.1991, 0.0000, 2.0071, 0.7854, 0.04, 0.04])

# set joints position
franka.ready_qpos = np.append(qr, [np.pi/2, np.pi/4, np.pi/4]) # virtual_finger, spinning pads
franka.set_qpos(franka.ready_qpos)
# position control
franka.control_dofs_position(franka.ready_qpos)

franka_MJCF.ready_qpos = qr
franka_MJCF.set_qpos(franka_MJCF.ready_qpos)
franka_MJCF.control_dofs_position(franka_MJCF.ready_qpos)

end_effector = franka.get_link('virtual_finger')
target_entity.set_pos(end_effector.get_pos())
target_entity.set_quat(end_effector.get_quat())

print(franka.ready_qpos)
scene.step()

In [ ]:
gs.quat_to_R(end_effector.get_quat())

In [12]:
# target_quat = np.array([0, 1, 0, 0])
# center = np.array([0.5, 0, 0.5])
# r = 0.1
# damping = 1e-6
# diag = damping * np.eye(6)
# target_pos = center + np.array([1, 0, 0]) * r
# init_q = franka.inverse_kinematics(
#     link = end_effector,
#     pos  = target_pos,
#     quat = target_quat,
# )
# franka.set_qpos(init_q)
# franka.control_dofs_position(init_q)
# for i in range(100):
#     target_entity.set_qpos(np.concatenate([target_pos, target_quat]))
#     scene.step()

# for i in range(0, 500):
#     target_pos = center + np.array([np.cos(i / 360 * np.pi), np.sin(i / 360 * np.pi), 0]) * r

#     target_entity.set_qpos(np.concatenate([target_pos, target_quat]))

#     # Position error.
#     error_pos = target_pos - end_effector.get_pos().cpu().numpy()

#     # Orientation error.
#     ee_quat = end_effector.get_quat().cpu().numpy()
#     error_quat = gs.transform_quat_by_quat(gs.inv_quat(ee_quat), target_quat)
#     error_rotvec = gs.quat_to_rotvec(error_quat)

#     error = np.concatenate([error_pos, error_rotvec])

#     # jacobian
#     jac = franka.get_jacobian(link=end_effector).cpu().numpy()
#     dq = jac.T @ np.linalg.solve(jac @ jac.T + diag, error)
#     q = franka.get_qpos().cpu().numpy() + dq

#     # control
#     franka.control_dofs_position(q)
#     scene.step()


## Pre-grasp Positioning

Now we'll move both robots to a position above the cube. We'll use inverse kinematics to calculate the joint positions needed to place the end-effector at the desired position and orientation. 


In [ ]:

# get the end-effector link
end_effector = franka.get_link('hand')
virtual_finger = franka.get_link('virtual_finger')
end_effector_MJCF = franka_MJCF.get_link('hand')
above_box = cube.get_pos().cpu().numpy() + np.array([-cube_size[2]/2 + 0.02, 0, 0.1 + cube_size[0]/2 - 0.02]) # 0.1 m above the grasp point
# move to pre-grasp pose
qpos = franka.inverse_kinematics(
    link = virtual_finger,
    pos  = above_box,
    rot_mask = [False, False, True]
)
qpos_MJCF = franka_MJCF.inverse_kinematics(
    link = end_effector_MJCF,
    pos  = above_box + np.array([0.0, 2.0, 0.1034]), # 0.1034 m is the distance between hand and fingers
    quat = np.array([0, 1, 0, 0]),
)
# gripper open pos
qpos_MJCF[fingers_dof] = 0.04
qpos[fingers_dof] = 0.04
print(qpos)


In [ ]:
path = franka.plan_path(
    qpos_goal     = qpos,
    num_waypoints = 200, # 2s duration
)
path_MJCF = franka_MJCF.plan_path(
    qpos_goal     = qpos_MJCF,
    num_waypoints = 200, # 2s duration
)

## Path Planning

Generate a smooth trajectory for each robot to reach the pre-grasp position.

In [19]:
# execute the planned path
for waypoint, waypoint_MJCF in zip(path, path_MJCF):
    franka.control_dofs_position(waypoint)
    franka_MJCF.control_dofs_position(waypoint_MJCF)
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()

# allow robot to reach the last waypoint
for i in range(100):
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()


## Gripper Positioning

Going down to align fingers with grasp point

In [24]:
side_box = above_box + np.array([0.0, 0.0, -0.1])


# reach
qpos_reach = franka.inverse_kinematics(
    link = virtual_finger,
    pos  = side_box,
    rot_mask = [False, False, True]
)
qpos_reach_MJCF = franka_MJCF.inverse_kinematics(
    link = end_effector_MJCF,
    pos  = side_box + np.array([0.0, 2.0, 0.1034]),
    quat = np.array([0, 1, 0, 0]),
)


## Reaching for the Object

Calculate and execute a path to move from the pre-grasp position down to the actual grasp position. Both robots move their end-effectors to the side of their respective cubes.


In [48]:
end_effector_dq = adapter.forward_kinematics(franka, end_effector, output_type='dq')
cube_bbox = adapter.get_bbox(cube)
qs, s_axes = planner.screw_from_bbox(cube_bbox)
q, s_axis = qs[1], s_axes[1]

In [49]:

trajectory_franka = planner.generate_screw_trajectory(initial_pose=end_effector_dq, q=q, s_axis=s_axis, output_type='pq')

In [ ]:
trajectory_franka

In [ ]:

for waypoint in trajectory_franka:
    franka.control_dofs_position(waypoint)
    franka_MJCF.control_dofs_position(waypoint_MJCF)
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()

# allow robot to reach the last waypoint
for i in range(100):
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()

In [26]:
franka.control_dofs_force(np.array([-2, -2]), fingers_dof)
franka_MJCF.set_qpos(qpos_reach_MJCF)
franka_MJCF.control_dofs_position(qpos_reach_MJCF[motors_dof], motors_dof)
franka_MJCF.control_dofs_force(np.array([-2, -2]), fingers_dof)

for i in range(100):
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()


## Grasping the Object

Now we'll close the grippers on both robots by applying force to the finger joints. This is where we can start observing differences:

1. The standard robot (without spinning pads) applies force only with its parallel-jaw gripper
2. The enhanced robot also activates its spinning pads, which distribute force more evenly across the object surface

The spinning pads reduce the chance of the object slipping by providing rolling contact rather than pure sliding contact.


In [ ]:

# lift
# move to pre-grasp pose
qpos = franka.inverse_kinematics(
    link = end_effector,
    pos  = above_box,
    quat = np.array([0, 1, 0, 0]),
)
qpos_MJCF = franka_MJCF.inverse_kinematics(
    link = end_effector_MJCF,
    pos  = above_box + np.array([0.0, 2.0, 0.0]),
    quat = np.array([0, 1, 0, 0]),
)

franka.control_dofs_position(qpos[:-4], motors_dof)
franka_MJCF.control_dofs_position(qpos_MJCF[:-2], motors_dof)

for i in range(200):
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()

## Lifting the Object

With the object grasped, we now lift both cubes upward. Watch carefully for any differences in stability:

- The standard gripper may show slight object rotation or slippage during movement
- The gripper with spinning pads typically maintains a more stable grasp by adapting to small forces with rolling action


In [ ]:
# move backward
# move to pre-grasp pose
qpos = franka.inverse_kinematics(
    link = end_effector,
    pos  = above_box + np.array([-0.1, 0.0, 0.1]),
    quat = np.array([0, 1, 0, 0]),
)
qpos_MJCF = franka_MJCF.inverse_kinematics(
    link = end_effector_MJCF,
    pos  = above_box + np.array([-0.1, 2.0, 0.1]),
    quat = np.array([0, 1, 0, 0]),
)

franka.control_dofs_position(qpos[:-4], motors_dof)
franka_MJCF.control_dofs_position(qpos_MJCF[:-2], motors_dof)

for i in range(200):
    target_entity.set_pos(end_effector.get_pos())
    target_entity.set_quat(end_effector.get_quat())
    scene.step()

## Transporting the Object

Finally, we move the robots backward while still grasping their cubes. This lateral movement introduces additional forces that can challenge grasp stability.

The differences between the two gripping methods should be most apparent during this phase:
- The standard gripper relies solely on friction to prevent slipping
- The gripper with spinning pads allows controlled rolling at the contact points, reducing the build-up of lateral forces

This demonstrates why spinning pads can be advantageous for delicate object manipulation tasks where maintaining precise grasp stability is critical.


## Conclusion

This tutorial demonstrated the difference between standard Franka grippers and those equipped with spinning pads. Key observations:

1. **Setup Complexity**: The spinning pad configuration requires additional DOFs (9-10) but provides enhanced manipulation capabilities
2. **Grasp Stability**: Spinning pads can improve stability by providing rolling contact instead of sliding contact
3. **Object Control**: During transport, the spinning pads help maintain more precise control of the object

For applications requiring precise manipulation or handling of delicate objects, the spinning pad configuration offers significant advantages despite the added complexity.

**Next Steps**:
- Try manipulating objects of different shapes and materials
- Experiment with different spinning pad velocities
- Implement closed-loop control using force feedback from the gripper
